# Comprehensive Project Challenge

## Summer Olympic Games, Medal Tables 1896-2012

### The Case

It´s your first day in a Data Science advisory firm and your boss asks you to produce the __official Summer Olympic Games Medal Tables for all Editions from 1896 to 2012__. <br><br>
All you can use is a dataset with raw data containing over 31,000 medals (__summer.csv__) and the official Medal Tables for the Editions 1996 and 1976 from Wikipedia. (__wik_1996.csv__, __wik_1976.csv__). Use the two official Medal Tables as a __reference__ to check whether your code produces the correct output! <br><br>
Your goal is to __minimize the divergence__ between your aggregated Medal Tables and the official Medal Tables. Let´s assume that the official number of Gold Medals for the United States in the Edition 1996 is 44 and your code produces 46. This is an absolute divergence of 2. <br> <br>
__Calculate the total absolute divergence for the Editions 1996 and 1976 (the "Score")!__ The __optimal Score is 0__! 

### Naive approach

In [1]:
import pandas as pd

In [2]:
summer = pd.read_csv("summer.csv")

In [3]:
summer76 = summer.loc[summer.Year == 1976]

In [4]:
summer76.head()

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
13900,1976,Montreal,Aquatics,Diving,"ALEINIK, Vladimir",URS,Men,10M Platform,Bronze
13901,1976,Montreal,Aquatics,Diving,"DIBIASI, Klaus",ITA,Men,10M Platform,Gold
13902,1976,Montreal,Aquatics,Diving,"LOUGANIS, Gregory",USA,Men,10M Platform,Silver
13903,1976,Montreal,Aquatics,Diving,"WILSON, Deborah Keplar",USA,Women,10M Platform,Bronze
13904,1976,Montreal,Aquatics,Diving,"VAYTSEKHOVSKAYA, Elena",URS,Women,10M Platform,Gold


In [5]:
mt76 = summer76.groupby(["Country", "Medal"]).Medal.count().unstack(fill_value = 0)
mt76.head()

Medal,Bronze,Gold,Silver
Country,,,
AUS,8,0,16
AUT,1,0,0
BEL,6,0,3
BER,1,0,0
BRA,3,0,0


In [6]:
mt76 = mt76.sort_values(["Gold", "Silver", "Bronze"], ascending = False)[["Gold", "Silver", "Bronze"]]
mt76.head(10)

Medal,Gold,Silver,Bronze
Country,,,
URS,113,93,79
GDR,99,51,42
USA,63,56,36
JPN,25,6,10
FRG,21,24,30
POL,18,29,26
NZL,17,1,9
HUN,14,6,35
SWE,9,1,0


In [7]:
pd.read_csv("wik_1976.csv")

,Rank,NOC,Gold,Silver,Bronze,Total
0,1,Soviet Union (URS),49,41,35,125
1,2,East Germany (GDR),40,25,25,90
2,3,United States (USA),34,35,25,94
3,4,West Germany (FRG),10,12,17,39
4,5,Japan (JPN),9,6,10,25
5,6,Poland (POL),7,6,13,26
6,7,Bulgaria (BUL),6,9,7,22
7,8,Cuba (CUB),6,4,3,13
8,9,Romania (ROU),4,9,14,27
9,10,Hungary (HUN),4,5,13,22


-> __Far away from the target!!!__

#### Fortunately, you could manage to get some useful information from Sports experts: <br>

Medals awarded in __Team Events__ (one medal for each member of the team) only count as __one Medal__. For example, the Basketball Team of the United States won the Gold Medal in the Edition 2012. In total __12 Basketball Athletes__ from the United States were awarded with a Gold Medal. For the official Medal Table 2012, this only counts as __one Gold Medal__ for the United States!<br> <br>
All Events with __5 or less than 5 medals__ shall be deemed __Singles Events__. All Events with __more than 5 medals__ shall be deemed __Team Events__. It frequently happens that 2 or 3 Athletes share the Bronze medal. Therefore, in total 4 or 5 medals are awarded in these Singles Events. All of these medals count for the official Medal Table! It also happens in Team Events that two Teams share the Bronze medal. Also in this case, in total 4 medals count for the official Medal Table (1 Gold, 1 Silver, 2 Bronze).
<br><br>
To identify all unique Events, the __Event Gender matters__! There are __Men__ Events, __Women__ Events and __Mixed__ Events. Assume that the following medals have been awarded in __Mixed Events__:
- the Event is marked with "__mixed__" or "__pairs__"
- all "__Equestrian__" Events
- all "__Sailing__" Events __before 1988__ (until and including 1984)
- the following medals (index labels) were awarded in __Badminton mixed Double Events__: [21773, 21782, 21776, 21785, 21770, 21779, 23703, 23712, 23706, 23715, 23709, 23700, 25720, 25729, 25723, 25732, 25726, 25717, 27727, 27736, 27730, 27739, 27724, 27733, 29784, 29785, 29786, 29787, 29788, 29789]

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Step 1: Getting Started

Inspect the three datasets and align wik_1996 and wik_1976 to the summer dataset! You will need this later when comparing your results with the official Medal Tables! 

In [9]:
wik_1996 = pd.read_csv('wik_1996.csv')
wik_1976 = pd.read_csv('wik_1976.csv')
summer = pd.read_csv('summer.csv')

In [10]:
wik_1976.head()

,Rank,NOC,Gold,Silver,Bronze,Total
0,1,Soviet Union (URS),49,41,35,125
1,2,East Germany (GDR),40,25,25,90
2,3,United States (USA),34,35,25,94
3,4,West Germany (FRG),10,12,17,39
4,5,Japan (JPN),9,6,10,25


In [11]:
wik_1996.head()

,Rank,Nation,Gold,Silver,Bronze,Total
0,1,United States (USA)*,44,32,25,101
1,2,Russia (RUS),26,21,16,63
2,3,Germany (GER),20,18,27,65
3,4,China (CHN),16,22,12,50
4,5,France (FRA),15,7,15,37


In [12]:
summer.head()

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver


In [13]:
summer[summer.isna().any(axis=1)]

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
29603,2012,London,Athletics,Athletics,Pending,NaN,Women,1500M,Gold
31072,2012,London,Weightlifting,Weightlifting,Pending,NaN,Women,63KG,Gold
31091,2012,London,Weightlifting,Weightlifting,Pending,NaN,Men,94KG,Silver
31110,2012,London,Wrestling,Wrestling Freestyle,"KUDUKHOV, Besik",NaN,Men,Wf 60 KG,Silver


In [14]:
summer.dropna(inplace=True)

In [15]:
wik_1976.head()

,Rank,NOC,Gold,Silver,Bronze,Total
0,1,Soviet Union (URS),49,41,35,125
1,2,East Germany (GDR),40,25,25,90
2,3,United States (USA),34,35,25,94
3,4,West Germany (FRG),10,12,17,39
4,5,Japan (JPN),9,6,10,25


In [16]:
wik_1976['Country'] = wik_1976.NOC.str.split(pat="(", expand=True).iloc[:,1].str.replace(")","").str.replace('*',"")

In [17]:
wik_1976.drop(columns=['NOC','Rank','Total'], inplace=True)

In [18]:
wik_1976.set_index('Country',inplace=True)

In [19]:
wik_1996['Country'] = wik_1996.Nation.str.split(pat='(', expand=True).iloc[:,1].str.replace(')','').str.replace('*','')

In [20]:
wik_1996.head()

,Rank,Nation,Gold,Silver,Bronze,Total,Country
0,1,United States (USA)*,44,32,25,101,USA
1,2,Russia (RUS),26,21,16,63,RUS
2,3,Germany (GER),20,18,27,65,GER
3,4,China (CHN),16,22,12,50,CHN
4,5,France (FRA),15,7,15,37,FRA


In [21]:
wik_1996.drop(columns=['Nation','Rank','Total'],inplace=True)

In [22]:
wik_1996.set_index('Country', inplace=True)

In [23]:
wik_1996.head()

,Gold,Silver,Bronze
Country,,,
USA,44,32,25
RUS,26,21,16
GER,20,18,27
CHN,16,22,12
FRA,15,7,15


In [24]:
wik_1976.head()

,Gold,Silver,Bronze
Country,,,
URS,49,41,35
GDR,40,25,25
USA,34,35,25
FRG,10,12,17
JPN,9,6,10


In [25]:
summer.head()

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver


### Step 2: Creating the Column Event_Gender

In a first step, we need to determine for each row / medal, whether the medal was awarded in a Men, Women or Mixed Event. <br>
The default assumption: The values in the new column Event_Gender are the same as in the column Gender (this is the Gender of the respective Athlete). Then, we need to identify Mixed Gender Events (inputs from the experts!).

In [26]:
summer['Event_Gender'] = summer.Gender

In [27]:
summer.head()

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal,Event_Gender
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold,Men
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver,Men
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze,Men
3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold,Men
4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver,Men


In [28]:
summer.loc[summer.Event.str.lower().str.contains('mixed'),'Event_Gender'].count()

38

In [29]:
summer.loc[summer.Event.str.lower().str.contains('pairs'),'Event_Gender'].count()

12

In [30]:
summer.loc[summer.Sport.str.lower().str.contains("equestrian"),'Event_Gender'].count()

939

All "Sailing" Events before 1988 have been mixed Events.

In [31]:
(summer.Sport.str.lower().str.contains('sailing') & (summer.Year < 1988)).sum()

755

The following medals (index labels) were awarded in Badminton mixed Double Events:

In [32]:
badm_mixed = [21773, 21782, 21776,21785, 21770, 21779,23703,23712,23706, 23715,23709,23700,25720,25729,25723,25732,25726,
              25717,27727,27736, 27730,27739,27724,27733, 29784, 29785,29786,29787,29788,29789]

In [33]:
mask1 = summer.Event.str.lower().str.contains('mixed')
mask2 = summer.Event.str.lower().str.contains('pairs')
mask3 = summer.Sport.str.lower().str.contains("equestrian")
mask4 = (summer.Sport.str.lower().str.contains('sailing') & (summer.Year < 1988))

In [34]:
summer.loc[mask1|mask2|mask3|mask4,'Event_Gender'] = 'X'

In [35]:
summer.loc[badm_mixed,'Event_Gender'] = 'X'

### Step 3: Identify all unique Events and count the amount of medals in each Event (new column Event_Medals)

In [36]:
summer['Event_Medal'] = summer.groupby(['Year','Sport','Discipline','Event','Event_Gender']).Medal.transform('count')

In [37]:
summer.head()

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal,Event_Gender,Event_Medal
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold,Men,2
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver,Men,2
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze,Men,3
3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold,Men,3
4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver,Men,3


In [38]:
summer.Event_Medal.value_counts().sort_index()

1         5
2        54
3      9228
4      1684
5        50
       ... 
73       73
74       74
76       76
82       82
116     116
Name: Event_Medal, Length: 66, dtype: int64

In [39]:
summer.loc[summer.Event_Medal == 5]

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal,Event_Gender,Event_Medal
1273,1908,London,Athletics,Athletics,"ARCHIBALD, Edward Blake",CAN,Men,Pole Vault,Bronze,Men,5
1274,1908,London,Athletics,Athletics,"JACOBS, Charles Sherman",USA,Men,Pole Vault,Bronze,Men,5
1275,1908,London,Athletics,Athletics,"SÖDERSTRÖM, Bruno",SWE,Men,Pole Vault,Bronze,Men,5
1276,1908,London,Athletics,Athletics,"COOKE, Edward Tiffin",USA,Men,Pole Vault,Gold,Men,5
1277,1908,London,Athletics,Athletics,"GILBERT, Alfred Carleten",USA,Men,Pole Vault,Gold,Men,5
7770,1948,London,Gymnastics,Artistic G.,"MOGYOROSI-KLENCS, Janos",HUN,Men,Vault,Bronze,Men,5
7771,1948,London,Gymnastics,Artistic G.,"PATAKI, Ferenc",HUN,Men,Vault,Bronze,Men,5
7772,1948,London,Gymnastics,Artistic G.,"SOTORNIK, Leo",TCH,Men,Vault,Bronze,Men,5
7773,1948,London,Gymnastics,Artistic G.,"AALTONEN, Paavo Johannes",FIN,Men,Vault,Gold,Men,5
7774,1948,London,Gymnastics,Artistic G.,"ROVE, Olavi Antero",FIN,Men,Vault,Silver,Men,5


Hint: The Columns "Year", "Sport", "Discipline", "Event", "Event_Gender" are relevant to group the summer DataFrame into unique events.

### Step 4: Identifying Team Events

All medals / rows, that were awarded in Events with more than 5 medals, shall be deemed Team Event Medals. (new column "Team")

In [40]:
summer['Team'] = pd.Series(np.where(summer.Event_Medal > 5,'Yes','No'))

In [41]:
summer.head()

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal,Event_Gender,Event_Medal,Team
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold,Men,2,No
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver,Men,2,No
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze,Men,3,No
3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold,Men,3,No
4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver,Men,3,No


In [42]:
summer.Team.value_counts()

Yes    20140
No     11017
Name: Team, dtype: int64

### Step 5: Removing Duplicated Medals in Team Events 

The subset for determining Duplicates shall be formed by the Columns "Year", "Sport", "Discipline", "Country", "Event", "Event_Gender", "Medal". Keep one Medal!

In [43]:
summer.reset_index(inplace=True)

In [44]:
summer.head()

,index,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal,Event_Gender,Event_Medal,Team
0,0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold,Men,2,No
1,1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver,Men,2,No
2,2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze,Men,3,No
3,3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold,Men,3,No
4,4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver,Men,3,No


In [45]:
singles = summer.loc[summer.Team == 'No'].copy()
singles.shape

(11017, 13)

In [46]:
team = summer.loc[summer.Team == 'Yes'].copy()
team.shape

(20140, 13)

In [47]:
team.drop_duplicates(subset = ['Year','Discipline','Sport','Country','Event','Event_Gender','Medal'],inplace=True)

In [48]:
team.shape

(3701, 13)

In [49]:
pd.concat([singles,team]).shape

(14718, 13)

In [50]:
summer_new = pd.concat([singles,team])

In [51]:
summer_new.head()

,index,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal,Event_Gender,Event_Medal,Team
0,0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold,Men,2,No
1,1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver,Men,2,No
2,2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze,Men,3,No
3,3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold,Men,3,No
4,4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver,Men,3,No


In [52]:
summer_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14718 entries, 0 to 31035
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         14718 non-null  int64 
 1   Year          14718 non-null  int64 
 2   City          14718 non-null  object
 3   Sport         14718 non-null  object
 4   Discipline    14718 non-null  object
 5   Athlete       14718 non-null  object
 6   Country       14718 non-null  object
 7   Gender        14718 non-null  object
 8   Event         14718 non-null  object
 9   Medal         14718 non-null  object
 10  Event_Gender  14718 non-null  object
 11  Event_Medal   14718 non-null  int64 
 12  Team          14718 non-null  object
dtypes: int64(3), object(10)
memory usage: 1.6+ MB


### Step 6: Creating the official Medal Table for all Editions

In [53]:
medal_tables = summer_new.groupby(['Year','Country','Medal']).Medal.count().unstack(fill_value=0)[['Gold','Silver','Bronze']]

In [54]:
medal_tables

Medal         Gold  Silver  Bronze
Year Country                      
1896 AUS         2       0       0
     AUT         2       1       2
     DEN         1       2       3
     FRA         5       4       2
     GBR         2       3       2
...            ...     ...     ...
2012 UGA         1       0       0
     UKR         6       4       9
     USA        46      28      32
     UZB         1       0       2
     VEN         1       0       0

[1157 rows x 3 columns]

### Step 7: Comparison with Wikipedia Medal Tables

In [55]:
medal_tables.head()

Medal         Gold  Silver  Bronze
Year Country                      
1896 AUS         2       0       0
     AUT         2       1       2
     DEN         1       2       3
     FRA         5       4       2
     GBR         2       3       2

In [56]:
agg1976 = medal_tables.loc[1976].sort_values(['Gold','Silver','Bronze'],ascending=False).copy()

In [57]:
score_76 = agg1976.sub(wik_1976).abs().sum().sum()

In [58]:
wik_1976.drop(index= '41 NOCs',inplace=True)

In [59]:
wik_1996.drop('79 nations',inplace=True)

In [60]:
agg1996 = medal_tables.loc[1996].sort_values(['Gold','Silver','Bronze'],ascending=False).copy()

In [61]:
score_96 = agg1996.sub(wik_1996).abs().sum().sum()

In [62]:
print(score_76,score_96)

0.0 0
